In [1]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error,r2_score
import warnings
import plotly.express as px
import plotly.graph_objects as go
import optuna
warnings.filterwarnings('ignore')

/home/khim3/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('./NFLX.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-02-05,262.000000,267.899994,250.029999,254.259995,254.259995,11896100
1,2018-02-06,247.699997,266.700012,245.000000,265.720001,265.720001,12595800
2,2018-02-07,266.579987,272.450012,264.329987,264.559998,264.559998,8981500
3,2018-02-08,267.079987,267.619995,250.000000,250.100006,250.100006,9306700
4,2018-02-09,253.850006,255.800003,236.110001,249.470001,249.470001,16906900


In [3]:
df = df[['Close']].rename(columns={'Close': 'close'})

In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error
import optuna
import plotly.graph_objects as go

class HoltWintersModel:
    def __init__(self, df):
        self.closedf = df.copy()
        self.best_params = None
        self.mae = None

    def optimize_holtwinter(self, seasonal_periods=60, n_trials=200):
        def objective_holtwinter(trial):
            alpha = trial.suggest_float('alpha', 0.01, 0.99)
            beta = trial.suggest_float('beta', 0.01, 0.99)
            gamma = trial.suggest_float('gamma', 0.01, 0.99)
            seasonal = trial.suggest_categorical('seasonal', ['add', 'multiplicative'])
            model = ExponentialSmoothing(
                self.closedf['close'],
                trend='add',
                seasonal=seasonal,
                seasonal_periods=seasonal_periods,
                damped_trend=True
            ).fit(
                smoothing_level=alpha,
                smoothing_trend=beta,
                smoothing_seasonal=gamma
            )
            predictions = model.fittedvalues
            return mean_absolute_error(self.closedf['close'], predictions.dropna())

        study = optuna.create_study(direction='minimize')
        study.optimize(objective_holtwinter, n_trials=n_trials)

        self.best_params = study.best_params
        self.mae = study.best_value
        return self.best_params, self.mae

    def fit_optimal_holtwinter_model(self, steps, seasonal_periods):
        model = ExponentialSmoothing(
            self.closedf['close'],
            trend='add',
            seasonal=self.best_params['seasonal'],
            seasonal_periods=seasonal_periods,
            damped_trend=True
        ).fit(
            smoothing_level=self.best_params['alpha'],
            smoothing_trend=self.best_params['beta'],
            smoothing_seasonal=self.best_params['gamma']
        )
        self.closedf['Holt_Winters_Optimal'] = model.fittedvalues
        return model.forecast(steps=steps)

    def plot_holtwinter_results(self):
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=self.closedf.index, y=self.closedf['Holt_Winters_Optimal'],
            mode='lines',
            name='Holt-Winters Predictions'
        ))
        fig.add_trace(go.Scatter(
            x=self.closedf.index, y=self.closedf['close'],
            mode='lines',
            name='Actual Close Price'
        ))
        fig.update_layout(
            title='Holt-Winters Predictions',
            xaxis_title='Date',
            yaxis_title='Price'
        )
        return fig

    def plot_forecast_holtwinter_results(self, forecast_values):
        future_index = pd.date_range(
            start=self.closedf.index[-1],
            periods=len(forecast_values) + 1,
            freq='D'
        )[1:]
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=future_index, y=forecast_values,
            mode='lines',
            name='Holt-Winters Forecast'
        ))
        fig.add_trace(go.Scatter(
            x=self.closedf.index, y=self.closedf['close'],
            mode='lines',
            name='Actual Close Price'
        ))
        fig.update_layout(
            title='Holt-Winters Forecast',
            xaxis_title='Date',
            yaxis_title='Price'
        )
        return fig


In [5]:
# Initialize the Holt-Winters model
hw_model = HoltWintersModel(df)

# Optimize hyperparameters
best_params, mae = hw_model.optimize_holtwinter(seasonal_periods=100, n_trials=100)
print(f"Best Parameters: {best_params}, MAE: {mae}")

# Fit the optimal model and forecast
forecast_steps = 30  # Forecast the next 30 days
forecast = hw_model.fit_optimal_holtwinter_model(steps=forecast_steps, seasonal_periods=150)

# Plot results
fig1 = hw_model.plot_holtwinter_results()

# Display the plots
fig1.show()


[I 2024-12-09 22:24:29,970] A new study created in memory with name: no-name-5a024a37-f1b1-4b96-b336-b2e4e4a2fecc
[I 2024-12-09 22:24:29,997] Trial 0 finished with value: 857.3580528180754 and parameters: {'alpha': 0.7033432007389826, 'beta': 0.9550225834530379, 'gamma': 0.8458275440894559, 'seasonal': 'multiplicative'}. Best is trial 0 with value: 857.3580528180754.
[I 2024-12-09 22:24:30,453] Trial 1 finished with value: 8.710303287219704 and parameters: {'alpha': 0.4716894693655305, 'beta': 0.051411265752792895, 'gamma': 0.057845267226395346, 'seasonal': 'multiplicative'}. Best is trial 1 with value: 8.710303287219704.
[I 2024-12-09 22:24:30,874] Trial 2 finished with value: 10.9969190035283 and parameters: {'alpha': 0.42218170736015836, 'beta': 0.8431655684979287, 'gamma': 0.3328111134985302, 'seasonal': 'multiplicative'}. Best is trial 1 with value: 8.710303287219704.
[I 2024-12-09 22:24:31,280] Trial 3 finished with value: 9.310668258339046 and parameters: {'alpha': 0.63654694687

Best Parameters: {'alpha': 0.8660667613489873, 'beta': 0.0506309747902092, 'gamma': 0.010448983473007378, 'seasonal': 'add'}, MAE: 7.372974375016422


In [7]:
# Step 1: Split the data
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Step 2: Initialize the Holt-Winters model with training data
hw_model = HoltWintersModel(train_data)

# Step 3: Optimize hyperparameters on the training data
best_params, mae = hw_model.optimize_holtwinter(seasonal_periods=60, n_trials=200)
print(f"Best Parameters: {best_params}, MAE: {mae}")

# Step 4: Fit the model with optimal parameters
forecast_steps = 10  # Number of steps to forecast = size of test data
forecast = hw_model.fit_optimal_holtwinter_model(steps=forecast_steps, seasonal_periods=60)

# Step 5: Evaluate the forecast against the test data
from sklearn.metrics import mean_squared_error, mean_absolute_error

test_mse = mean_squared_error(test_data['close'], forecast)
test_mae = mean_absolute_error(test_data['close'], forecast)
print(f"Test MSE: {test_mse:.3f}, Test MAE: {test_mae:.3f}")

# Step 6: Plot the results
fig1 = hw_model.plot_holtwinter_results()  # Plot training and fitted values
fig2 = hw_model.plot_forecast_holtwinter_results(forecast)  # Plot the forecast
fig1.show()
fig2.show()

[I 2024-12-09 22:26:47,279] A new study created in memory with name: no-name-9bdc44ad-7576-412c-b38c-617411dd1b08
[I 2024-12-09 22:26:47,323] Trial 0 finished with value: 16.36191793634367 and parameters: {'alpha': 0.08961355607539605, 'beta': 0.7263136511350597, 'gamma': 0.02433520963855585, 'seasonal': 'add'}. Best is trial 0 with value: 16.36191793634367.
[I 2024-12-09 22:26:47,345] Trial 1 finished with value: 90.75034789972587 and parameters: {'alpha': 0.4106294853145964, 'beta': 0.7880473304132699, 'gamma': 0.7662215765348163, 'seasonal': 'add'}. Best is trial 0 with value: 16.36191793634367.
[I 2024-12-09 22:26:47,366] Trial 2 finished with value: 819.4043526341543 and parameters: {'alpha': 0.03092254678919784, 'beta': 0.3787887812749781, 'gamma': 0.3559343552469776, 'seasonal': 'multiplicative'}. Best is trial 0 with value: 16.36191793634367.
[I 2024-12-09 22:26:47,412] Trial 3 finished with value: 14.879167727024473 and parameters: {'alpha': 0.220385156101307, 'beta': 0.306214

Best Parameters: {'alpha': 0.923437892283658, 'beta': 0.01000142246994833, 'gamma': 0.010721826166243969, 'seasonal': 'multiplicative'}, MAE: 7.321369733359654


ValueError: Found input variables with inconsistent numbers of samples: [202, 10]